# Speech to Text module

This section obtains all the audio files

In [20]:
import os
import soundfile
import wave
import pandas as pd
from google.cloud import speech_v1p1beta1 as speech

Cut the audio, keeping the first minute 

In [9]:
path = 'audioSales2/'
for file in os.listdir(path):
    if(file.endswith('.wav')):
        nameSolo = file.rsplit('.', 1)[0]
        ifile = wave.open(path + file)
        framesIn10Seconds = ifile.getframerate()*59
        if framesIn10Seconds > ifile.getnframes():
            framesIn10Seconds = ifile.getnframes()        
        
        ofile = wave.open(path + '/1_min/' + nameSolo + '_1Min.wav', "w") 
        ofile.setparams(ifile.getparams())
        ofile.setnframes(framesIn10Seconds)
        ofile.writeframes(ifile.readframes(framesIn10Seconds))
        print("Cutting " + file + "to 1 minute of duration")        
        ifile.close()
        ofile.close()

Cutting I_29693[1].wavto 1 minute of duration
Cutting I_29711[1].wavto 1 minute of duration
Cutting I_29806[1].wavto 1 minute of duration
Cutting I_29836[1].wavto 1 minute of duration


Converting the audio from 8 to 16 bits

In [10]:
path = 'audioSales2/1_min/'
for file in os.listdir(path):
    if(file.endswith('.wav')):
        nameSolo = file.rsplit('.', 1)[0]
        print(path + nameSolo )
        data, samplerate = soundfile.read(path + file)                

        soundfile.write(path + '16_bits/' + nameSolo + '_16BIT.wav', data, samplerate, subtype='PCM_16')
        print("converting " + file + "to 16 - bit")
   

audioSales2/1_min/I_29693[1]_1Min
converting I_29693[1]_1Min.wavto 16 - bit
audioSales2/1_min/I_29711[1]_1Min
converting I_29711[1]_1Min.wavto 16 - bit
audioSales2/1_min/I_29806[1]_1Min
converting I_29806[1]_1Min.wavto 16 - bit
audioSales2/1_min/I_29836[1]_1Min
converting I_29836[1]_1Min.wavto 16 - bit


Generating a dataframe with the transcription

In [33]:
path = 'audioSales2/1_min/16_bits/'  
client = speech.SpeechClient()
wholeConversation = ''
audioColumn = []
audioData = []
for file in os.listdir(path):
    if(file.endswith('.wav')):
        nameSolo = file.rsplit('.', 1)[0]
        print(path + nameSolo )
        with open(path + file, 'rb') as audio_file:
                content = audio_file.read()

        audio = speech.types.RecognitionAudio(content=content)

        config = speech.types.RecognitionConfig(
            encoding=speech.enums.RecognitionConfig.AudioEncoding.LINEAR16,
            sample_rate_hertz=8000,
            language_code='es-MX',
            enable_speaker_diarization=True)

        print('Waiting for operation to complete...')
        response = client.recognize(config, audio)

        # The transcript within each result is separate and sequential per result.
        # However, the words list within an alternative includes all the words
        # from all the results thus far. Thus, to get all the words with speaker
        # tags, you only have to take the words list from the last result:
        result = response.results[-1]

        words_info = result.alternatives[0].words 
        # Printing out the output:
        print(f'Transcription of the audio file:{path + file}')
        for word_info in words_info:
            wholeConversation = wholeConversation + ' ' + word_info.word  
        audioColumn.append(path +  file)
        audioData.append(wholeConversation)
        wholeConversation = '' 
audioDF = pd.DataFrame(audioData, audioColumn)
audioDF    

audioSales2/1_min/16_bits/I_29693[1]_1Min_16BIT
Waiting for operation to complete...
Transcription of the audio file:audioSales2/1_min/16_bits/I_29693[1]_1Min_16BIT.wav
audioSales2/1_min/16_bits/I_29711[1]_1Min_16BIT
Waiting for operation to complete...
Transcription of the audio file:audioSales2/1_min/16_bits/I_29711[1]_1Min_16BIT.wav
audioSales2/1_min/16_bits/I_29806[1]_1Min_16BIT
Waiting for operation to complete...
Transcription of the audio file:audioSales2/1_min/16_bits/I_29806[1]_1Min_16BIT.wav
audioSales2/1_min/16_bits/I_29836[1]_1Min_16BIT
Waiting for operation to complete...
Transcription of the audio file:audioSales2/1_min/16_bits/I_29836[1]_1Min_16BIT.wav


,0
audioSales2/1_min/16_bits/I_29693[1]_1Min_16BIT.wav,Hola qué tal Muy buenos días Gracias por llam...
audioSales2/1_min/16_bits/I_29711[1]_1Min_16BIT.wav,Hola qué tal Muy buenos días Gracias por comu...
audioSales2/1_min/16_bits/I_29806[1]_1Min_16BIT.wav,Hola que tal Buenos días bienvenida mejorcomp...
audioSales2/1_min/16_bits/I_29836[1]_1Min_16BIT.wav,Hola buenos días Gracias por comunicarse a nu...


In [41]:
audioDF[0][3]

' Hola buenos días Gracias por comunicarse a nuestra Mejor compra TV la tiene Paola Cervantes con quien tengo el gusto mucho gusto señores el verdad Estela mucho gusto señores tela se comunica directamente a la línea de los sartenes flavor Stone es correcto pero que sí señor le stelle entonces Coméntame Cómo es la promoción que está observando en este momento de los sartenes flavor Stone exactamente señores está cargando su promoción especial en la cual acabé de entrar su llamada mientras se termina de cargar esta promoción comenté mi señora Estela qué fue lo que más le agradó o le gustó de eso sartenes'